# Agent Action and Agent Finish

In [1]:
%load_ext dotenv
%dotenv

In [81]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.vectorstores import Chroma

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool

from importlib.metadata import distributions
from langchain import hub

from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate

from platform import python_version

from langchain.agents import create_tool_calling_agent, AgentExecutor

# Tools

1. Wiki-tool
2. Retriever
3. Python custom fun

In [19]:
wikipeida_tool = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper())

In [20]:
vectorstore = Chroma(persist_directory='./intro-ds-vectorstore',
                     embedding_function= OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_type = 'mmr',
                                     search_kwargs = {'k':3,
                                                      'lambda_mult':0.7})

retriever_tool = create_retriever_tool(retriever = retriever,
                                       name = 'Introduction-to-Data-and-Data-Science-Course-Lectures', #Adding - to avoid BadRequest Error
                                       description = '''For any questions regarding the introduction to Data and Data Science course,
                                       you must use this tool.''')

In [51]:
@tool
def get_python_version() -> str:
    '''Returns the current Python version running in the environment.'''
    return python_version()


@tool
def get_installed_packages() -> str:
    '''Returns all installed pip packages with their versions.'''
    return "\n".join(
        f"{d.metadata['Name']}=={d.version}" for d in distributions()
    )


In [52]:
tools = [wikipeida_tool,retriever_tool,get_python_version, get_installed_packages]

# Chat

In [53]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  temperature = 0,
                  model_kwargs={'seed':365})

/Users/aungkaungpyaepaing/PycharmProjects/LangChain/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3610: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


# Memory

In [89]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True,    output_key="output")

# Chat-Prompt-Template

In [90]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

new_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI agent."),
    ("user", "{chat_history}"),
    ("user", "{input}"),
    ("ai", "{agent_scratchpad}")
])

/Users/aungkaungpyaepaing/PycharmProjects/LangChain/.venv/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [91]:
chat_prompt_template.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


# Creating Agent

In [92]:
agent = create_tool_calling_agent(llm = chat,
                                  tools = tools,
                                  prompt = new_prompt)

In [93]:
agent_executor = AgentExecutor(agent = agent,
                               tools = tools,
                               memory = memory,
                               verbose = True,
                               return_intermediate_steps=True)

In [94]:
response = agent_executor.invoke({'input':"What's the latest version of Python?"})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.11.2The latest version of Python running in this environment is 3.11.2.

> Finished chain.
The latest version of Python running in this environment is 3.11.2.


In [95]:
response = agent_executor.invoke({"input":'''
I’m currently following the Introduction to Data and Data Science course.
Does it include like i have to use specific version of Python?
'''})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `Introduction-to-Data-and-Data-Science-Course-Lectures` with `{'query': 'Python version requirement'}`


More importantly, it will be sufficient for your need to create quick and accurate analyses. However, if your theoretical preparation is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on

As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end

Great! We hope we gave you a good idea about the l

In [96]:
response = agent_executor.invoke({"input": '''
To follow that course what kind of packages do i need to install?
In addition, do i have those suggested packages already installed in my system?
'''})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `Introduction-to-Data-and-Data-Science-Course-Lectures` with `{'query': 'required packages'}`


More importantly, it will be sufficient for your need to create quick and accurate analyses. However, if your theoretical preparation is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on

Among the many applications we have plotted, we can say there is an increasing amount of software designed for working with big data such as Apache Hadoop, Apache Hbase, and Mongo DB. In terms of big data, Hadoop is the name that must stick with you. Hadoop is listed as a software in the sense that it is a collection of programs, but don’t imagine it as a nice-looking application

Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software 

In [98]:
USER_COLOR = "\033[92m"   # Green
AI_COLOR = "\033[96m"     # Cyan
RESET = "\033[0m"

history = memory.load_memory_variables({})["chat_history"]

for message in history:
    if message.type == "human":
        print(f"{USER_COLOR}User: {message.content}{RESET}")
    elif message.type == "ai":
        print(f"{AI_COLOR}AI: {message.content}{RESET}")


User: What's the latest version of Python?
AI: The latest version of Python running in this environment is 3.11.2.
User: 
I’m currently following the Introduction to Data and Data Science course.
Does it include like i have to use specific version of Python?

AI: The Introduction to Data and Data Science course does not specify a particular version of Python that you need to use. The course emphasizes the importance of knowing a programming language like Python to create specific, ad-hoc tools for each project you are working on. 

R and Python are highlighted as the two most popular tools across all columns in data science. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations but are adaptable and can solve a wide variety of business and data-related problems from beginning to the end.

So, you can use the version of Python that you 